#### student ID : 20141445

##### import librarys

In [1]:
import matplotlib.pyplot as plt
import numpy as np

##### Load dataset and define constants

In [2]:
file_data		= "mnist_train.csv"
handle_file	= open(file_data, "r")
data        		= handle_file.readlines()
handle_file.close()

file_data_test		= "mnist_test.csv"
handle_file_test	= open(file_data_test, "r")
data_test        		= handle_file_test.readlines()
handle_file_test.close()

size_row	= 28    # height of the image
size_col  	= 28    # width of the image

num_image	= len(data)
num_image_test = len(data_test)
count       	= 0     # count for the number of images
count_test       	= 0     # count for the number of images

##### Define functions

In [3]:
# return average of input images
def average(images):
    return normalize(np.sqrt(np.sum(np.asarray(images)**2,axis=0)))

##### read csv file

In [4]:
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:
    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    list_label[count]       = label
    list_image[:, count]    = im_vector    
    count += 1
    
list_image_test  = np.empty((size_row * size_col, num_image_test), dtype=float)
list_label_test  = np.empty(num_image_test, dtype=int)

for line in data_test:
    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    list_label_test[count_test]       = label
    list_image_test[:, count_test]    = im_vector    
    count_test += 1

##### train data whitening

In [5]:
image_mean = np.mean(list_image, axis=1)
image_std = np.std(list_image, axis=1)
image_std[image_std == 0] = 1

list_image = (np.transpose(list_image) - image_mean)
list_image = np.transpose(list_image/image_std)

##### test data whitening

In [6]:
image_mean_test = np.mean(list_image_test, axis=1)
image_std_test = np.std(list_image_test, axis=1)
image_std_test[image_std_test == 0] = 1

list_image_test = (np.transpose(list_image_test) - image_mean_test)
list_image_test = np.transpose(list_image_test/image_std_test)

1. Compute an optimal model parameter using the training dataset for each classifier $f_d(x, w)$

$$ f_d(x; w) = w_0 * 1 + w_1 * g_1 + w_2 * g_2 + ... + w_k * g_k $$

where $w = (w_0, w_1, ... , w_k)$ and the basis function $g_k$ is defined by the inner product of random vector $r_k$ and input vector $x$. 

In [7]:
MEAN = 0
STD = 1
M = size_row * size_col
K = 1024

def make_random_matrix():
    randoms = np.random.normal(MEAN, STD, K * M)
    return np.reshape(randoms, (K, M))

random_matrix = make_random_matrix()
def multiply_random_A(matrix_A):
    matrix_A = np.matmul(random_matrix, matrix_A)
    matrix_A = np.transpose(matrix_A)
    matrix_A = np.insert(matrix_A,0,1,axis=1) # insert 1 on first of every row
    matrix_A = np.where(matrix_A < 0, 0, matrix_A) # make megative value to zero
    return matrix_A

matrix_A = multiply_random_A(list_image)

coefficients = list()
for label_number in range(10):
    matrix_b = np.array(list(map(lambda y: -1 if y != label_number else 1 ,list_label)))
    coefficients.append(np.matmul(np.linalg.pinv(matrix_A), matrix_b))

2. Compute (1) true positive rate, (2) error rate using (1) training dataset and (2) testing dataset.

In [8]:
def classify(picture, coefficients):
    picture = np.matmul(random_matrix, picture)
    picture = np.insert(0,1,picture)
    picture = np.where(picture < 0, 0, picture)
    max_sim = - 9999999
    max_label = -1
    for i in range(10):
        similarity = np.matmul(picture, coefficients[i])
        if similarity > max_sim:
            max_sim = similarity
            max_label = i
    return max_label

true_count = np.zeros((10))
error_count = np.zeros((10))
# train set
for i in range(count):    
    if list_label[i] == classify(list_image[:,i],coefficients):
        true_count[list_label[i]] += 1
    else:
        error_count[list_label[i]] += 1

true_count_test = np.zeros((10))
error_count_test = np.zeros((10))
# test set
for i in range(count_test):  
    if list_label_test[i] == classify(list_image_test[:,i],coefficients):
        true_count_test[list_label_test[i]] += 1
    else:
        error_count_test[list_label_test[i]] += 1

        
t1 = np.sum(true_count)
e1 = np.sum(error_count)
t2 = np.sum(true_count_test)
e2 = np.sum(error_count_test)
# print result
print("#1 train set")
print("true positive rate: " + str(round(t1/(t1+e1),5)))
print("error rate: " + str(round(e1/(t1+e1),5)))
print("\n#2 test set")
print("true positive rate: " + str(round(t2/(t2+e2),5)))
print("error rate: " + str(round(e2/(t2+e2),5)))

#1 train set
true positive rate: 0.91728
error rate: 0.08272

#2 test set
true positive rate: 0.912
error rate: 0.088
